In [404]:
from pandas import read_csv
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [405]:
import pandas as pd
from data import citylearn_challenge_2022_phase_1 as competition_data
import os.path as osp

nsload_data_path = osp.join(osp.dirname(competition_data.__file__), "load_data.csv")
nsload_data = pd.read_csv(nsload_data_path)

# for i in range(5):
#     plt.plot(nsload_data["non_shiftable_load"][i*8760 : 8760 + i*8760])
#     plt.show()

In [406]:
#"month", "hour", "outdoor_dry_bulb", "outdoor_dry_bulb_temperature_predicted_6h", "outdoor_relative_humidity", "outdoor_relative_humidity_predicted_6h", "diffuse_solar_irradiance", "diffuse_solar_irradiance_predicted_6h", "diffuse_solar_irradiance_predicted_24h", "direct_solar_irradiance", "direct_solar_irradiance_predicted_6h", "direct_solar_irradiance_predicted_24h", "electricity_pricing", "electricity_pricing_predicted_6h", "electricity_pricing_predicted_24h"

selected_params = ["outdoor_relative_humidity", "outdoor_dry_bulb_temperature_predicted_6h", "outdoor_dry_bulb", "outdoor_dry_bulb_temperature_predicted_6h", "hour", "diffuse_solar_irradiance", "diffuse_solar_irradiance_predicted_24h", "month", "diffuse_solar_irradiance_predicted_6h", "direct_solar_irradiance_predicted_6h", "direct_solar_irradiance_predicted_24h", "direct_solar_irradiance", "non_shiftable_load_future"]
nsload_data = nsload_data[selected_params]

In [407]:
nsload_data

,outdoor_relative_humidity,outdoor_dry_bulb_temperature_predicted_6h,outdoor_dry_bulb,outdoor_dry_bulb_temperature_predicted_6h,hour,diffuse_solar_irradiance,diffuse_solar_irradiance_predicted_24h,month,diffuse_solar_irradiance_predicted_6h,direct_solar_irradiance_predicted_6h,direct_solar_irradiance_predicted_24h,direct_solar_irradiance,non_shiftable_load_future
0,79.0,19.4,20.1,19.4,1.0,0.0,0.0,8.0,201.0,444.0,0.0,0.0,0.834600
1,78.0,21.1,19.7,21.1,2.0,0.0,0.0,8.0,420.0,592.0,0.0,0.0,0.838167
2,78.0,22.2,19.3,22.2,3.0,0.0,0.0,8.0,554.0,491.0,0.0,0.0,1.478433
3,78.0,21.7,18.9,21.7,4.0,0.0,0.0,8.0,778.0,734.0,0.0,0.0,1.256200
4,81.0,22.2,18.3,22.2,5.0,0.0,0.0,8.0,903.0,811.0,0.0,0.0,1.869517
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43785,81.0,20.0,21.1,20.0,18.0,218.0,231.0,7.0,0.0,0.0,320.0,289.0,2.730183
43786,84.0,20.1,20.6,20.1,19.0,26.0,27.0,7.0,0.0,0.0,143.0,130.0,0.935833
43787,81.0,19.7,21.1,19.7,20.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,2.155017
43788,79.0,19.3,21.7,19.3,21.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,2.316400


In [408]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# nsload_data = nsload_data[0:8760]
print(nsload_data)

df_load_scaled = nsload_data.iloc[: , :-1]
df_load_scaled = sc.fit_transform(df_load_scaled)

print(df_load_scaled)
print(df_load_scaled.shape)

sc2 = StandardScaler()
df_load_scaled_y = sc2.fit_transform(nsload_data[["non_shiftable_load_future"]])
print(df_load_scaled_y.shape)

       outdoor_relative_humidity  outdoor_dry_bulb_temperature_predicted_6h  \
0                           79.0                                       19.4   
1                           78.0                                       21.1   
2                           78.0                                       22.2   
3                           78.0                                       21.7   
4                           81.0                                       22.2   
...                          ...                                        ...   
43785                       81.0                                       20.0   
43786                       84.0                                       20.1   
43787                       81.0                                       19.7   
43788                       79.0                                       19.3   
43789                       76.0                                       18.9   

       outdoor_dry_bulb  outdoor_dry_bulb_temperatu

In [409]:
hops = 6
no_records = df_load_scaled_y.shape[0]
no_cols = len(selected_params) - 1
X_train = []
y_train = []
for i in range(hops, no_records):
    X_train.append(df_load_scaled[i-hops:i])
    y_train.append(df_load_scaled_y[i][0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

(43784, 6, 12)
(43784,)


In [410]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

In [411]:
X_train_reshaped.shape

(43784, 6, 12)

In [412]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model=Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(hops,no_cols)))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [413]:
model.fit(X_train_reshaped, y_train, validation_split=0, epochs=300, batch_size=128, shuffle=True, workers=24, use_multiprocessing=True)

Epoch 1/300
343/343 [==============================] - 23s 59ms/step - loss: 0.8612
Epoch 2/300
343/343 [==============================] - 20s 59ms/step - loss: 0.8245
Epoch 3/300
343/343 [==============================] - 22s 63ms/step - loss: 0.8103
Epoch 4/300
343/343 [==============================] - 21s 62ms/step - loss: 0.8016
Epoch 5/300
343/343 [==============================] - 21s 62ms/step - loss: 0.7969
Epoch 6/300
343/343 [==============================] - 22s 63ms/step - loss: 0.7917
Epoch 7/300
343/343 [==============================] - 21s 62ms/step - loss: 0.7868
Epoch 8/300
343/343 [==============================] - 21s 62ms/step - loss: 0.7856
Epoch 9/300
343/343 [==============================] - 21s 62ms/step - loss: 0.7831
Epoch 10/300
343/343 [==============================] - 21s 62ms/step - loss: 0.7805
Epoch 11/300
343/343 [==============================] - 21s 61ms/step - loss: 0.7778
Epoch 12/300
343/343 [==============================] - 21s 62ms/step - lo

KeyboardInterrupt: 

In [403]:
from pickle import dump
model.save("non_shiftable_load_model.h5")
dump(sc, open("non_shiftable_load_scaler.pkl", "wb"))
dump(sc2, open("non_shiftable_load_scaler_result.pkl", "wb"))

In [60]:
# units = dimensionality of the output space


def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape,
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

demo_model = create_RNN(hidden_units=2, dense_units=1, input_shape=(3,1), activation=['relu', 'relu'])